# Batch Serving with Ray Serve Example

Demonstrates Ray Serve deployment for scalable, parallel model serving.

In [1]:
import ray
from ray import serve
import joblib
import pandas as pd
from fastapi import FastAPI
from pydantic import BaseModel

ray.init(ignore_reinit_error=True)
serve.start(detached=True)

## Define FastAPI App and Ray Serve Deployment

In [2]:
class BridgeInput(BaseModel):
    features: dict

app = FastAPI()

@serve.deployment(route_prefix="/predict")
class BridgeModelDeployment:
    def __init__(self):
        self.model = joblib.load('../models/trained/model.joblib')
    async def __call__(self, request):
        data = await request.json()
        X = pd.DataFrame(data)
        preds = self.model.predict_proba(X)[:, 1].tolist()
        return preds

BridgeModelDeployment.bind()

## Query Ray Serve Endpoint (after deployment)
```
import requests
resp = requests.post('http://127.0.0.1:8000/predict', json=payload)
print(resp.json())
```